In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from biopsykit.utils.dataframe_handling import multi_xs

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from fau_colors import register_cmaps
register_cmaps()

sns.set_theme(context="talk", style="white", palette="faculties", font_scale=1.2)

%matplotlib widget

In [5]:
df = pd.read_pickle("../full_df.pkl")

In [6]:
df = df.rename(index={"specifity":"specificity"})
df

data  \
algorithm type algorithm modality        stage  subject metric                                
DL             LSTM      ACT             3stage 0027    accuracy                  73.951049   
                                                        bed_interval_end        1131.000000   
                                                        bed_interval_start        99.000000   
                                                        f1                        65.924768   
                                                        getup_latency              0.000000   
...                                                                                     ...   
ML             XGBoost   ACT + HRV + RRV Binary 6807    sleep_onset_latency        5.000000   
                                                        specificity               94.650206   
                                                        total_sleep_duration    1004.000000   
                                                        wake_after_sleep_onset   447.000000   
                                                        wake_onset               982.000000   

                                                                                    race  \
algorithm type algorithm modality        stage  subject metric                             
DL             LSTM      ACT             3stage 0027    accuracy                   White   
                                                        bed_interval_end           White   
                                                        bed_interval_start         White   
                                                        f1                         White   
                                                        getup_latency              White   
...                                                                                  ...   
ML             XGBoost   ACT + HRV + RRV Binary 6807    sleep_onset_latency     Hispanic   
                                                        specificity             Hispanic   
                                                        total_sleep_duration    Hispanic   
                                                        wake_after_sleep_onset  Hispanic   
                                                        wake_onset              Hispanic   

                                                                                gender  \
algorithm type algorithm modality        stage  subject metric                           
DL             LSTM      ACT             3stage 0027    accuracy                female   
                                                        bed_interval_end        female   
                                                        bed_interval_start      female   
                                                        f1                      female   
                                                        getup_latency           female   
...                                                                                ...   
ML             XGBoost   ACT + HRV + RRV Binary 6807    sleep_onset_latency       male   
                                                        specificity               male   
                                                        total_sleep_duration      male   
                                                        wake_after_sleep_onset    male   
                                                        wake_onset                male   

                                                                                PSG_quality  \
algorithm type algorithm modality        stage  subject metric                                
DL             LSTM      ACT             3stage 0027    accuracy                        6.0   
                                                        bed_interval_end                6.0   
                                                        bed_interval_start              6.0   
        

# 5 stage

In [7]:
df_5 = multi_xs(df, ["5stage"], level="stage")
df_5_mcc = multi_xs(df_5, ["mcc"],level="metric")[["data"]].astype(float)
df_5 = multi_xs(df_5, ["accuracy", "f1", "precision", "recall", "specificity"],level="metric")[["data"]].astype(float)

In [8]:
df_5_mean = df_5.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
#df_5_mean = df_5_mean.unstack("modality")["data"]["mean"][["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + EDR"]]
df_5_mean = df_5_mean.unstack("metric")["data"].round(decimals=1).applymap('{:,.1f}'.format).applymap('{:0>4}'.format)

In [9]:
df_5_mcc = df_5_mcc.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
df_5_mcc = df_5_mcc.unstack("metric")["data"].round(decimals=2).applymap('{:,.2f}'.format).applymap('{:0>4}'.format)

In [10]:
#df_5_mean.sort_values(by=["accuracy"], axis=0, ascending=False).sort_index(level="modality", sort_remaining=False)

In [11]:
df_5_mean = pd.DataFrame(
    "$" + df_5_mean["median"].astype(str) + " (" + df_5_mean["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_5_mean = df_5_mean.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [12]:
df_5_mcc = pd.DataFrame(
    "$" + df_5_mcc["median"].astype(str) + " (" + df_5_mcc["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_5_mcc = df_5_mcc.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [13]:
df_5_mean = df_5_mean.join(df_5_mcc).reindex(["accuracy", "f1", "mcc", "precision", "recall", "specificity"], axis=1)

In [14]:
df_5_mean.style.to_latex("5stage.tex")

In [15]:
df_5_mean

metric                              accuracy             f1            mcc  \
modality        algorithm                                                    
ACT             LSTM           $56.1 (13.1)$  $46.5 (13.2)$  $0.37 (0.18)$   
                TCN            $56.1 (13.3)$  $46.8 (13.2)$  $0.37 (0.18)$   
                AdaBoost       $55.0 (14.1)$  $45.9 (14.3)$  $0.33 (0.19)$   
                MLP            $54.9 (13.5)$  $46.2 (13.6)$  $0.34 (0.18)$   
                Random Forest  $55.1 (13.6)$  $45.6 (13.7)$  $0.34 (0.19)$   
                SVM            $55.0 (14.5)$  $45.8 (14.8)$  $0.33 (0.19)$   
                XGBoost        $55.2 (14.3)$  $45.9 (14.4)$  $0.34 (0.19)$   
ACT + HRV       LSTM           $64.9 (14.9)$  $59.6 (15.5)$  $0.50 (0.17)$   
                TCN            $61.4 (12.5)$  $55.3 (13.6)$  $0.44 (0.16)$   
                AdaBoost       $56.4 (12.5)$  $48.7 (12.3)$  $0.36 (0.16)$   
                MLP            $56.8 (12.3)$  $48.6 (13.2)$  $0.36 (0.16)$   
                Random Forest  $56.9 (12.5)$  $48.5 (13.1)$  $0.36 (0.17)$   
                SVM            $56.3 (12.6)$  $47.0 (13.2)$  $0.35 (0.16)$   
                XGBoost        $57.2 (12.0)$  $49.3 (12.1)$  $0.37 (0.17)$   
ACT + HRV + EDR LSTM           $63.7 (13.1)$  $57.2 (14.8)$  $0.49 (0.16)$   
                TCN            $62.0 (12.6)$  $56.6 (14.6)$  $0.46 (0.18)$   
                AdaBoost       $57.2 (12.2)$  $50.5 (13.0)$  $0.37 (0.15)$   
                MLP            $57.9 (13.0)$  $50.9 (13.9)$  $0.38 (0.16)$   
                Random Forest  $57.5 (12.8)$  $50.0 (14.1)$  $0.38 (0.17)$   
                SVM            $56.8 (12.2)$  $48.3 (12.4)$  $0.36 (0.16)$   
                XGBoost        $58.2 (12.2)$  $51.1 (13.7)$  $0.38 (0.16)$   
ACT + HRV + RRV LSTM           $65.7 (11.9)$  $60.1 (12.4)$  $0.51 (0.16)$   
                TCN            $62.4 (12.7)$  $57.9 (13.1)$  $0.46 (0.18)$   
                AdaBoost       $61.9 (12.1)$  $56.7 (12.9)$  $0.44 (0.17)$   
                MLP            $62.7 (12.4)$  $56.6 (13.5)$  $0.46 (0.15)$   
                Random Forest  $62.3 (12.4)$  $55.7 (12.6)$  $0.45 (0.17)$   
                SVM            $60.5 (12.1)$  $53.4 (12.9)$  $0.42 (0.16)$   
                XGBoost        $63.2 (12.0)$  $57.2 (12.8)$  $0.46 (0.15)$   

metric                             precision         recall    specificity  
modality        algorithm                                                   
ACT             LSTM           $45.5 (14.9)$  $56.1 (13.1)$  $86.8 (08.4)$  
                TCN            $45.5 (14.1)$  $56.1 (13.3)$  $86.5 (08.0)$  
                AdaBoost       $44.1 (15.2)$  $55.0 (14.1)$  $85.6 (08.3)$  
                MLP            $43.6 (14.6)$  $54.9 (13.5)$  $85.0 (08.3)$  
                Random Forest  $43.6 (13.8)$  $55.1 (13.6)$  $85.3 (08.3)$  
                SVM            $43.8 (14.5)$  $55.0 (14.5)$  $85.6 (09.3)$  
                XGBoost        $43.8 (14.7)$  $55.2 (14.3)$  $85.2 (08.2)$  
ACT + HRV       LSTM           $65.8 (13.8)$  $64.9 (14.9)$  $88.4 (06.5)$  
                TCN            $60.5 (14.5)$  $61.4 (12.5)$  $86.5 (07.4)$  
                AdaBoost       $52.8 (14.0)$  $56.4 (12.5)$  $85.5 (07.1)$  
                MLP            $53.6 (13.1)$  $56.8 (12.3)$  $85.6 (07.5)$  
                Random Forest  $52.0 (14.8)$  $56.9 (12.5)$  $85.6 (07.4)$  
                SVM            $47.5 (14.2)$  $56.3 (12.6)$  $85.6 (07.9)$  
                XGBoost        $53.4 (13.8)$  $57.2 (12.0)$  $86.0 (06.6)$  
ACT + HRV + EDR LSTM           $60.9 (13.9)$  $63.7 (13.1)$  $88.9 (06.3)$  
                TCN            $62.1 (14.2)$  $62.0 (12.6)$  $86.5 (07.3)$  
                AdaBoost       $55.4 (17.2)$  $57.2 (12.2)$  $85.4 (07.7)$  
                MLP            $56.0 (14.7)$  $57.9 (13.0)$  $85.4 (07.4)$  
                Random Forest  $53.4 (16.5)$  $57.5 (12.8)$  $86.3 (07.9)$  
                SVM            $52.1 (15.5

# 3 stage

In [16]:
df_3 = multi_xs(df, ["3stage"], level="stage")
df_3_mcc = multi_xs(df_3, ["mcc"],level="metric")[["data"]].astype(float)
df_3 = multi_xs(df_3, ["accuracy", "f1", "precision", "recall", "specificity"],level="metric")[["data"]].astype(float)

In [17]:
df_3_mean = df_3.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
#df_3_mean = df_3_mean.unstack("modality")["data"]["mean"][["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + EDR"]]
df_3_mean = df_3_mean.unstack("metric")["data"].round(decimals=1).applymap('{:,.1f}'.format).applymap('{:0>4}'.format)

In [18]:
df_3_mcc = df_3_mcc.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
df_3_mcc = df_3_mcc.unstack("metric")["data"].round(decimals=2).applymap('{:,.2f}'.format).applymap('{:0>4}'.format)

In [19]:
#df_3_mean.sort_values(by=["accuracy"], axis=0, ascending=False).sort_index(level="modality", sort_remaining=False)

In [20]:
df_3_mean = pd.DataFrame(
    "$" + df_3_mean["median"].astype(str) + " (" + df_3_mean["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_3_mean = df_3_mean.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [21]:
df_3_mcc = pd.DataFrame(
    "$" + df_3_mcc["median"].astype(str) + " (" + df_3_mcc["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_3_mcc = df_3_mcc.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [22]:
df_3_mean = df_3_mean.join(df_3_mcc).reindex(["accuracy", "f1", "mcc", "precision", "recall", "specificity"], axis=1)

In [23]:
df_3_mean.style.to_latex("3stage.tex")

In [24]:
df_3_mean

metric                              accuracy             f1            mcc  \
modality        algorithm                                                    
ACT             LSTM           $71.5 (11.4)$  $65.3 (11.7)$  $0.47 (0.21)$   
                TCN            $59.2 (09.4)$  $53.1 (08.6)$  $0.19 (0.16)$   
                AdaBoost       $70.0 (12.7)$  $64.1 (13.7)$  $0.44 (0.23)$   
                MLP            $70.1 (12.0)$  $64.2 (13.7)$  $0.44 (0.23)$   
                Random Forest  $70.1 (13.3)$  $64.1 (13.8)$  $0.44 (0.23)$   
                SVM            $69.8 (11.9)$  $63.9 (13.9)$  $0.44 (0.23)$   
                XGBoost        $70.3 (12.6)$  $64.4 (13.8)$  $0.45 (0.23)$   
ACT + HRV       LSTM           $77.7 (10.7)$  $75.5 (12.2)$  $0.60 (0.19)$   
                TCN            $75.6 (12.4)$  $73.3 (13.2)$  $0.55 (0.21)$   
                AdaBoost       $70.6 (11.3)$  $65.2 (12.8)$  $0.45 (0.20)$   
                MLP            $70.9 (11.1)$  $65.8 (11.8)$  $0.46 (0.20)$   
                Random Forest  $71.2 (11.3)$  $65.9 (12.3)$  $0.46 (0.20)$   
                SVM            $70.2 (11.5)$  $64.5 (12.9)$  $0.44 (0.20)$   
                XGBoost        $71.0 (11.4)$  $66.3 (11.6)$  $0.46 (0.19)$   
ACT + HRV + EDR LSTM           $77.5 (11.5)$  $75.6 (13.1)$  $0.59 (0.20)$   
                TCN            $71.4 (10.6)$  $67.4 (11.5)$  $0.48 (0.17)$   
                AdaBoost       $71.0 (11.0)$  $66.2 (12.1)$  $0.46 (0.20)$   
                MLP            $71.7 (11.4)$  $67.3 (12.7)$  $0.47 (0.20)$   
                Random Forest  $71.6 (11.5)$  $66.8 (12.7)$  $0.47 (0.21)$   
                SVM            $70.8 (10.9)$  $65.4 (12.5)$  $0.46 (0.20)$   
                XGBoost        $72.1 (11.1)$  $67.7 (11.9)$  $0.48 (0.20)$   
ACT + HRV + RRV LSTM           $79.0 (09.8)$  $77.7 (11.3)$  $0.62 (0.20)$   
                TCN            $73.1 (10.2)$  $70.6 (11.5)$  $0.51 (0.19)$   
                AdaBoost       $74.1 (10.2)$  $72.3 (11.4)$  $0.53 (0.19)$   
                MLP            $75.6 (10.3)$  $73.9 (11.3)$  $0.55 (0.17)$   
                Random Forest  $74.5 (10.2)$  $72.6 (12.3)$  $0.55 (0.18)$   
                SVM            $72.7 (10.7)$  $68.6 (11.6)$  $0.50 (0.18)$   
                XGBoost        $75.5 (10.4)$  $74.1 (11.7)$  $0.56 (0.17)$   

metric                             precision         recall    specificity  
modality        algorithm                                                   
ACT             LSTM           $64.7 (11.1)$  $71.5 (11.4)$  $84.9 (12.5)$  
                TCN            $51.1 (08.5)$  $59.2 (09.4)$  $67.5 (12.3)$  
                AdaBoost       $63.7 (10.6)$  $70.0 (12.7)$  $82.9 (13.2)$  
                MLP            $63.8 (10.2)$  $70.1 (12.0)$  $83.1 (12.5)$  
                Random Forest  $64.0 (10.5)$  $70.1 (13.3)$  $83.5 (13.2)$  
                SVM            $63.5 (11.0)$  $69.8 (11.9)$  $83.1 (12.2)$  
                XGBoost        $64.0 (10.3)$  $70.3 (12.6)$  $83.4 (12.9)$  
ACT + HRV       LSTM           $80.4 (09.2)$  $77.7 (10.7)$  $87.4 (08.2)$  
                TCN            $77.1 (10.0)$  $75.6 (12.4)$  $84.0 (11.0)$  
                AdaBoost       $68.2 (11.9)$  $70.6 (11.3)$  $83.3 (12.3)$  
                MLP            $69.8 (11.8)$  $70.9 (11.1)$  $83.4 (12.0)$  
                Random Forest  $68.6 (13.0)$  $71.2 (11.3)$  $83.5 (12.2)$  
                SVM            $64.0 (09.7)$  $70.2 (11.5)$  $83.3 (12.4)$  
                XGBoost        $69.9 (12.1)$  $71.0 (11.4)$  $83.7 (12.7)$  
ACT + HRV + EDR LSTM           $79.8 (09.5)$  $77.5 (11.5)$  $87.3 (09.1)$  
                TCN            $72.8 (11.4)$  $71.4 (10.6)$  $83.6 (10.2)$  
                AdaBoost       $71.3 (11.9)$  $71.0 (11.0)$  $83.2 (12.3)$  
                MLP            $70.5 (13.9)$  $71.7 (11.4)$  $83.0 (11.6)$  
                Random Forest  $71.2 (12.8)$  $71.6 (11.5)$  $83.5 (11.2)$  
                SVM            $67.2 (13.3

# 2 stage

In [25]:
df_2 = multi_xs(df, ["Binary"], level="stage")
df_2_mcc = multi_xs(df_2, ["mcc"],level="metric")[["data"]].astype(float)
df_2 = multi_xs(df_2, ["accuracy", "f1", "precision", "recall", "specificity"],level="metric")[["data"]].astype(float)

In [26]:
df_2_mean = df_2.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
#df_2_mean = df_2_mean.unstack("modality")["data"]["mean"][["ACT", "ACT + HRV", "ACT + HRV + RRV", "ACT + HRV + EDR"]]
df_2_mean = df_2_mean.unstack("metric")["data"].round(decimals=1).applymap('{:,.1f}'.format).applymap('{:0>4}'.format)

In [27]:
df_2_mcc = df_2_mcc.groupby(level=["algorithm type", "algorithm", "modality", "stage", "metric"]).agg([np.median, lambda x: np.percentile(x, 75) - np.percentile(x, 25)])
df_2_mcc = df_2_mcc.unstack("metric")["data"].round(decimals=2).applymap('{:,.2f}'.format).applymap('{:0>4}'.format)

In [28]:
#df_2_mean.sort_values(by=["accuracy"], axis=0, ascending=False).sort_index(level="modality", sort_remaining=False)

In [29]:
df_2_mean = pd.DataFrame(
    "$" + df_2_mean["median"].astype(str) + " (" + df_2_mean["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_2_mean = df_2_mean.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [30]:
df_2_mcc = pd.DataFrame(
    "$" + df_2_mcc["median"].astype(str) + " (" + df_2_mcc["<lambda_0>"].astype(str) + ")$"
).droplevel([0,3]).swaplevel(0,1).sort_index(level=0)#.sort_values(by="modality")
df_2_mcc = df_2_mcc.reindex(["LSTM", "TCN", "AdaBoost", "MLP", "Random Forest", "SVM", "XGBoost"], level="algorithm")

In [31]:
df_2_mean = df_2_mean.join(df_2_mcc).reindex(["accuracy", "f1", "mcc", "precision", "recall", "specificity"], axis=1)

In [32]:
df_2_mean.style.to_latex("2stage.tex")

In [33]:
df_2_mean.style.to_latex("2stage.tex")

In [34]:
df_2_mean

metric                              accuracy             f1            mcc  \
modality        algorithm                                                    
ACT             LSTM           $85.1 (13.2)$  $89.2 (10.7)$  $0.62 (0.27)$   
                TCN            $84.9 (14.2)$  $89.5 (11.0)$  $0.62 (0.27)$   
                AdaBoost       $83.9 (14.2)$  $89.2 (12.3)$  $0.57 (0.29)$   
                MLP            $84.1 (13.8)$  $89.2 (11.9)$  $0.59 (0.28)$   
                Random Forest  $84.4 (14.2)$  $89.0 (12.1)$  $0.59 (0.28)$   
                SVM            $83.1 (13.9)$  $88.8 (11.9)$  $0.57 (0.27)$   
                XGBoost        $84.2 (14.0)$  $89.1 (12.2)$  $0.58 (0.28)$   
ACT + HRV       LSTM           $87.3 (10.3)$  $91.2 (09.2)$  $0.68 (0.25)$   
                TCN            $84.9 (11.6)$  $89.6 (10.5)$  $0.61 (0.26)$   
                AdaBoost       $84.3 (14.1)$  $89.2 (12.3)$  $0.60 (0.26)$   
                MLP            $84.0 (13.8)$  $89.4 (11.8)$  $0.60 (0.26)$   
                Random Forest  $84.9 (14.1)$  $89.5 (12.4)$  $0.60 (0.27)$   
                SVM            $84.0 (13.8)$  $89.2 (12.0)$  $0.58 (0.25)$   
                XGBoost        $84.7 (14.0)$  $89.2 (11.9)$  $0.61 (0.26)$   
ACT + HRV + EDR LSTM           $86.5 (10.7)$  $90.1 (08.9)$  $0.67 (0.26)$   
                TCN            $85.3 (12.2)$  $89.9 (09.6)$  $0.63 (0.23)$   
                AdaBoost       $84.3 (14.0)$  $89.2 (12.0)$  $0.60 (0.26)$   
                MLP            $84.9 (13.0)$  $89.1 (11.6)$  $0.62 (0.25)$   
                Random Forest  $85.1 (13.6)$  $89.4 (12.0)$  $0.61 (0.27)$   
                SVM            $84.2 (13.8)$  $89.2 (11.8)$  $0.60 (0.27)$   
                XGBoost        $84.7 (13.5)$  $89.2 (11.6)$  $0.61 (0.26)$   
ACT + HRV + RRV LSTM           $87.9 (09.3)$  $91.0 (08.5)$  $0.71 (0.22)$   
                TCN            $86.3 (11.6)$  $90.1 (09.7)$  $0.66 (0.24)$   
                AdaBoost       $85.9 (13.0)$  $90.1 (09.9)$  $0.63 (0.25)$   
                MLP            $86.1 (11.6)$  $90.1 (09.7)$  $0.66 (0.22)$   
                Random Forest  $86.0 (13.2)$  $90.1 (10.0)$  $0.64 (0.24)$   
                SVM            $85.1 (12.8)$  $89.9 (10.5)$  $0.62 (0.24)$   
                XGBoost        $86.4 (12.3)$  $90.4 (09.6)$  $0.66 (0.23)$   

metric                             precision         recall    specificity  
modality        algorithm                                                   
ACT             LSTM           $88.7 (13.9)$  $94.3 (10.2)$  $70.9 (33.3)$  
                TCN            $88.7 (14.6)$  $94.1 (10.3)$  $70.7 (32.5)$  
                AdaBoost       $85.1 (15.6)$  $96.5 (09.0)$  $60.1 (32.4)$  
                MLP            $84.7 (15.3)$  $96.5 (08.1)$  $60.4 (32.2)$  
                Random Forest  $84.9 (15.7)$  $96.2 (08.4)$  $60.9 (32.8)$  
                SVM            $84.5 (15.6)$  $96.6 (08.1)$  $57.4 (32.7)$  
                XGBoost        $85.1 (16.2)$  $96.3 (08.4)$  $60.5 (33.3)$  
ACT + HRV       LSTM           $89.6 (13.4)$  $94.8 (06.7)$  $76.4 (28.7)$  
                TCN            $86.9 (14.5)$  $95.5 (07.2)$  $64.4 (28.5)$  
                AdaBoost       $86.1 (15.8)$  $95.7 (10.0)$  $64.8 (33.8)$  
                MLP            $86.6 (15.0)$  $95.7 (08.9)$  $66.1 (34.4)$  
                Random Forest  $86.1 (16.1)$  $95.4 (08.6)$  $64.4 (35.6)$  
                SVM            $86.0 (15.5)$  $96.1 (09.2)$  $61.8 (34.5)$  
                XGBoost        $86.6 (15.8)$  $95.3 (09.6)$  $65.9 (33.5)$  
ACT + HRV + EDR LSTM           $89.0 (14.4)$  $94.1 (08.7)$  $73.0 (26.6)$  
                TCN            $87.0 (14.2)$  $95.3 (07.1)$  $67.2 (28.3)$  
                AdaBoost       $86.7 (13.8)$  $95.5 (10.6)$  $67.0 (34.3)$  
                MLP            $87.1 (14.6)$  $95.4 (08.4)$  $67.9 (34.3)$  
                Random Forest  $86.3 (15.2)$  $95.4 (08.9)$  $64.3 (34.7)$  
                SVM            $86.6 (14.3